In [6]:
### Feature Selection
import sys
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell'
assert sys.version_info >= (3, 5)  # make sure we have Python 3.5+
from pyspark.sql import SparkSession, functions, types
spark = SparkSession.builder.appName('IoT').getOrCreate()
spark.sparkContext.setLogLevel('WARN')
assert spark.version >= '2.3'  # make sure we have Spark 2.3+
import os
import pandas as pd
import csv
import numpy as np
import random
import glob
import matplotlib
import matplotlib.pyplot as plt
import random
from sklearn.feature_selection import SelectKBest, chi2

get_ipython().magic('matplotlib inline')
df=pd.read_csv('/Users/liyifan/PycharmProjects/CMPT733/Project/rw_etl.csv/part-00000-14639349-6824-4673-b158-2b90e5e7a25c-c000.csv').sort_values(['TimeStamp'], ascending=True).reset_index()

df.drop(['::[scararobot]Ax_J1.PositionCommand','::[scararobot]Ax_J1.TorqueFeedback','::[scararobot]Ax_J2.PositionCommand','::[scararobot]Ax_J2.TorqueFeedback','::[scararobot]Ax_J3.TorqueFeedback','::[scararobot]Ax_J6.TorqueFeedback','::[scararobot]ScanTimeAverage','::[scararobot]Ax_J6.PositionCommand','::[scararobot]Ax_J3.PositionCommand','index'], axis=1, inplace=True)
df['TimeStamp']=pd.to_datetime(df['TimeStamp'])
# df.drop(['index'], axis=1, inplace=True)
pd.set_option('display.max_columns', None)  
df.head(5)
df.iloc[:,1:21]=df.iloc[:,1:21].abs()
df['Total']= df.select_dtypes(include=['float64','float32']).apply(lambda row: np.sum(row),axis=1)
x=df.iloc[:,1:20].values

y=df.iloc[0:,20:21].values
y=y.flatten()
y = np.array(y).astype(int)
SelBest = SelectKBest(chi2, k=15)
features=SelBest.fit_transform(x,y)
print(SelBest.get_support())
choosed_features=list(SelBest.get_support())
print(features)
sumed=np.sum(features,axis=1)
f_df=df.iloc[0:,0:1]
for index,fea in enumerate(choosed_features):
    if(fea==True):
        f_df[str(index)]=df.iloc[0:,index]


[ True False  True  True False  True  True False  True  True False  True
  True  True  True  True  True  True  True]
[[  0.         0.         0.       ...   0.         0.         0.      ]
 [ 59.35408   12.6       55.206806 ...  55.206806   0.17553  100.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 ...
 [  8.27028    8.3       62.732567 ...  62.732567   8.22891  100.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  8.607437   8.        61.62711  ...  61.62711    6.955    100.      ]]


,TimeStamp,0,2,3,5,6,8,9,11,12,13,14,15,16,17,18
0,2016-07-22 14:02:54.084,2016-07-22 14:02:54.084,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2016-07-22 14:02:54.087,2016-07-22 14:02:54.087,0.367542,12.600000,0.016180,2.800000,0.630125,57.100002,2.709248,11.600000,311.991241,169.906830,0.175530,59.354080,55.206806,0.175530
2,2016-07-22 14:02:54.134,2016-07-22 14:02:54.134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2016-07-22 14:02:54.137,2016-07-22 14:02:54.137,0.373410,12.800000,0.015765,2.700000,0.630938,57.000000,2.640736,11.200000,307.978271,177.130295,0.158390,60.671337,55.189659,0.158390
4,2016-07-22 14:02:54.184,2016-07-22 14:02:54.184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2016-07-22 14:02:54.187,2016-07-22 14:02:54.187,0.378360,12.800000,0.015171,2.800000,0.634250,56.700001,2.544077,10.500000,303.798065,184.259003,0.141250,61.989475,55.172352,0.141250
6,2016-07-22 14:02:54.234,2016-07-22 14:02:54.234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2016-07-22 14:02:54.237,2016-07-22 14:02:54.237,0.377280,12.900001,0.015251,2.700000,0.638625,57.200001,2.463347,10.100000,299.452850,191.289093,0.124100,63.313774,55.155697,0.124100
8,2016-07-22 14:02:54.284,2016-07-22 14:02:54.284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2016-07-22 14:02:54.287,2016-07-22 14:02:54.287,0.382176,13.000000,0.015587,2.800000,0.640688,57.700001,2.336529,9.800000,294.944946,198.216721,0.106960,64.632004,55.139259,0.106960
